# LightSSS：轻量级的仿真快照系统
**LightSSS: Light-weight Simulation Snapshot System**

依靠 Difftest，我们已经解决了“找到错误发生的位置”这一问题，但当我们定位到错误位置后，仍然需要手动打开波形输出并重新运行仿真，才能得到调试所需的波形文件。这一过程非常耗时，特别是对于运行时间较长的工作负载（如 SPEC CPU）。

Thanks to DiffTest, we have already addressed the issue of "How to locate where the error occurred". However, after finding the failure point, we still need to manually enable the waveform output and rerun the simulation to obtain the waveform files needed for debugging. This process is very time-consuming, especially for long-running workloads like SPEC CPU.

为了进一步提高效率，我们需要一种可以在错误发生时自动恢复到错误发生前的状态，并且从那里开始生成波形文件的方法。换句话说，我们需要一种快照机制。

To further improve efficiency, we need a method that can automatically restore to the state before the error occurred and generate waveform files from that state when an error is detected. In other words, we need a snapshot mechanism.

LightSSS 正是为了解决这个问题设计的，它依靠 Linux 的 `fork()` 系统调用和写时复制机制，实现了低成本的仿真快照。

LightSSS is designed to solve this problem. It utilizes Linux's `fork()` system call and the copy-on-write mechanism to achieve low-cost simulation snapshots.

![LightSSS](../images/02-functional/03-lightSSS/lightSSS-overall-en.png)

使用 `fork()` 的另一个好处是其具有良好的可扩展性，该系统调用自然地对整个进程空间进行快照，因此任何外部模型都可以轻松地集成进来。

Another advantage of using `fork()` is its good scalability. This system call naturally snapshots the entire process space, allowing any external models to be easily integrated.

使用 LightSSS 非常简单。只需在运行 emu 时添加 `--enable-fork` 参数即可启用该功能：

To use LightSSS is straightforward. Simply add the `--enable-fork` parameter when running emu to enable the feature:

In [ ]:
%%bash
source ../env.sh
cd ${NOOP_HOME}

mkdir -p build
rm -f ./build/*.vcd

$(get_asset emu-precompile/emu-alu-err) \
    -i $(get_asset workload/hello-riscv64-xs.bin) \
    --diff $(get_asset emu-precompile/riscv64-nemu-interpreter-so) \
    --enable-fork \
    2> /dev/null || true

echo -n "Dump wave: "
realpath ./build/*.vcd

如果您看到“the oldest checkpoint start to dump wave and dump nemu log...”，则表示 LightSSS 已生效。此后，模拟将从现有的最新快照重新开始并记录波形。

If you see "the oldest checkpoint start to dump wave and dump nemu log...", LightSSS is active. The simulation will then restart from the latest snapshot and record waveforms.